## Visualizations

In [1]:
import pandas as pd
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [2]:
by_isrc = pd.read_csv('data/checkpoint/by_isrc_oldest.csv')

In [3]:
by_isrc.head().T

,0,1,2,3,4
isrc,AEA040700577,AEA040700578,AEA040700579,AEA040700580,AEA040700581
genres,"['j-pop', 'jazz', 'pop']","['groove', 'j-pop', 'jazz', 'pop']","['groove', 'j-pop', 'jazz', 'pop']","['groove', 'j-pop', 'jazz', 'pop']","['groove', 'j-pop', 'jazz', 'pop', 'pop']"
name,Bala Wala Chi,Houdou Nisbi,Nafs Al Sheghlat,Yalla Kichou Barra,Ma Tfel
artists,Ziad Rahbani,Ziad Rahbani,Ziad Rahbani,Ziad Rahbani,Ziad Rahbani
album,Houdou Nisbi,Houdou Nisbi,Houdou Nisbi,Houdou Nisbi,Houdou Nisbi
release_date,1985-01-01,1985-01-01,1985-01-01,1985-01-01,1985-01-01
release_date_precision,day,day,day,day,day
uri,spotify:track:0fylgLeNObjVvwhd8caHqX,spotify:track:0yMFpBNCYXqwwOAg23bC8a,spotify:track:6G8l1kI8QlTD0UDIak5F8H,spotify:track:21g76Lq5Jg4QvfTDvi4PlH,spotify:track:0pKxrkFh8fxPKpkO29MYmi
spotify_id,0fylgLeNObjVvwhd8caHqX,0yMFpBNCYXqwwOAg23bC8a,6G8l1kI8QlTD0UDIak5F8H,21g76Lq5Jg4QvfTDvi4PlH,0pKxrkFh8fxPKpkO29MYmi
chart_power,NaN,NaN,NaN,NaN,NaN


## Find the biggest genres

In [4]:
remove_genres = ["classical", "jazz", "folk", "french", "turkish", "gospel", "samba", "piano", "mpb", "sertanejo", "pagode", "sleep", "forro", "malay", "anime", 
                 "j-idol", "comedy", "mandopop", "cantopop", "show-tunes", "emo", "romance", "j-dance", "chill", "world-music", "iranian", "idm", "metalcore", 
                 "hardstyle", "opera", "k-pop", "j-pop", "j-rock", "happy"]

In [42]:
df = pd.read_csv('data/DE/data-neu.csv')
df.drop_duplicates(inplace=True)

In [55]:
df.head(2).T

,0,1
name,That's No Way To Get Along,I Got A Problem
artists,Robert Wilkins,Albert Collins
album,The Original Rolling Stone,Frostbite
release_date,1980-01-01,1980-02-05
release_date_precision,day,day
chart_power,NaN,NaN
spotify_id,5JKdsNzhJGpoaKBFZpOKQ3,6e2PhVePvqe7w4VNBTslef
uri,spotify:track:5JKdsNzhJGpoaKBFZpOKQ3,spotify:track:6e2PhVePvqe7w4VNBTslef
popularity,22,15
danceability,0.427,0.76


In [7]:
ct = pd.crosstab(df['isrc'], df['genres'])
# ct.reset_index(inplace=True)
ct = ct.applymap(lambda x: 1 if x > 1 else x)
ct.reset_index(inplace=True)

In [8]:
# drop genres of the DataFrame
ct = ct.set_index('isrc').drop(columns=remove_genres)

In [9]:
# find all songs which are not assigned to genre anymore and remove them
print(f'Shape before removing songs. ct: {ct.shape}; df: {df.shape}')
remove_songs = ct[ct.T.sum() == 0].index.to_list()
ct.drop(index=remove_songs, inplace=True)
df.set_index('isrc', inplace=True)
df.drop(index=remove_songs, inplace=True)
df.reset_index(inplace=True)
print(f'Shape after removing songs. ct: {ct.shape}; df: {df.shape}')

Shape before removing songs. ct: (427265, 76); df: (863828, 24)
Shape after removing songs. ct: (316490, 76); df: (712299, 24)


In [10]:
ct_sum = ct.sum()

In [11]:
ct_sum_df = pd.DataFrame(ct_sum)

In [12]:
#ct_sum_df.drop(index='isrc', inplace=True)

In [13]:
ct_sum_df.rename(columns={0: 'count'}, inplace=True)

In [14]:
sorted_genres = ct_sum_df.sort_values(by='count', ascending=False).index.to_list()

In [15]:
ct_sum_df.sort_values(by='count', ascending=False)[:50]

,count
genres,
pop,118762
rock,93935
german,37752
punk,24275
metal,15761
country,15546
blues,12211
hard-rock,11659
soul,11099


Biggest genres is pop.

Count occurences of the top genres per year

In [16]:
by_isrc.shape

(427258, 25)

In [17]:
by_isrc_year = by_isrc.copy()
by_isrc_year = by_isrc_year[['year', 'isrc']]
by_isrc_year = by_isrc_year.merge(ct, on=['isrc'], how='left')
by_isrc_year.set_index('year', inplace=True)
by_isrc_year = by_isrc_year[sorted_genres[:10]]
by_isrc_year = by_isrc_year.groupby('year').sum()
by_isrc_year.index = by_isrc_year.index.astype(str)

In [18]:
by_isrc_year

,pop,rock,german,punk,metal,country,blues,hard-rock,soul,british
year,,,,,,,,,,
1980,11523.0,8459.0,2248.0,2165.0,737.0,1804.0,1231.0,974.0,1391.0,917.0
1981,10277.0,8302.0,2471.0,2455.0,823.0,1711.0,1068.0,910.0,1359.0,690.0
1982,10359.0,7874.0,2185.0,2417.0,803.0,44.0,994.0,1034.0,1367.0,831.0
1983,10033.0,8292.0,3433.0,2366.0,1114.0,1483.0,971.0,1150.0,1034.0,630.0
1984,12202.0,8837.0,3047.0,2419.0,1532.0,1265.0,914.0,1328.0,835.0,825.0
1985,11372.0,8721.0,3443.0,2251.0,1612.0,1640.0,753.0,1135.0,809.0,730.0
1986,11430.0,9258.0,3824.0,2322.0,1807.0,1628.0,865.0,1182.0,766.0,983.0
1987,12310.0,10706.0,4458.0,2510.0,2293.0,1876.0,1279.0,1417.0,1014.0,1316.0
1988,13809.0,11045.0,6465.0,2736.0,2380.0,1930.0,1722.0,1221.0,1186.0,1515.0


## Development of features over the years

In [91]:
# remove songs that have not the correct genres
df = by_isrc.copy()
print(df.shape)
df.set_index('isrc', inplace=True)
remove_help = []
df_index = df.index.to_list()
for song_isrc in remove_songs:
    if song_isrc in df_index:
        remove_help.append(song_isrc)
try:
    df.drop(index=remove_help, inplace=True)
except Exception as e:
    print(e)
    
df.reset_index(inplace=True)
print(df.shape)

(427258, 25)
(316490, 25)


In [ ]:
save = df.copy()
df['year_month'] = 0
for index, song in df.iterrows():
    df.loc[index,'year_month'] = song['year']
    if song['release_date_precision'] == 'month':
        df.loc[index,'year_month'] += int(song['release_date'][5:7]) / 12


In [100]:
df.head(2).T

,0,1
isrc,AEA040700577,AEA040700578
genres,"['j-pop', 'jazz', 'pop']","['groove', 'j-pop', 'jazz', 'pop']"
name,Bala Wala Chi,Houdou Nisbi
artists,Ziad Rahbani,Ziad Rahbani
album,Houdou Nisbi,Houdou Nisbi
release_date,1985-01-01,1985-01-01
release_date_precision,day,day
uri,spotify:track:0fylgLeNObjVvwhd8caHqX,spotify:track:0yMFpBNCYXqwwOAg23bC8a
spotify_id,0fylgLeNObjVvwhd8caHqX,0yMFpBNCYXqwwOAg23bC8a
chart_power,NaN,NaN


In [160]:
per_year_features = ['year', 'popularity', 'danceability', 'energy',
                     'key', 'loudness', 'mode', 'speechiness',
                     'acousticness', 'instrumentalness', 'liveness',
                     'valence', 'tempo', 'duration_ms',
                     'time_signature']
per_year_agg = {
    "popularity": 'mean',
    'danceability': 'mean', 
    'energy': 'mean',
    'key': 'mean', 
    'loudness': 'mean', 
    'mode': 'mean', 
    'speechiness': 'mean',
    'acousticness': 'mean', 
    'instrumentalness': 'mean', 
    'liveness': 'mean',
    'valence': 'mean', 
    'tempo': 'mean', 
    'duration_ms': 'mean',
    'time_signature': 'mean',
    'count': 'count'
}

songs_per_year = df.copy()
songs_per_year = songs_per_year[per_year_features]
songs_per_year['count'] = -1
songs_per_year = songs_per_year.groupby("year").agg(per_year_agg)
songs_per_year.reset_index(inplace=True)
songs_per_year['duration_min'] = songs_per_year['duration_ms'] / (1000 * 60)

In [148]:
songs_per_year.head(10).T

,0,1,2,3,4,5,6,7,8,9
year,1980.000000,1981.000000,1982.000000,1983.000000,1984.000000,1985.000000,1986.000000,1987.000000,1988.000000,1989.000000
popularity,7.937137,8.046343,8.037993,8.095217,8.452725,8.251112,8.073371,8.014330,7.558946,7.448754
danceability,0.519827,0.522204,0.529009,0.517713,0.519658,0.513429,0.502094,0.500046,0.500834,0.502221
energy,0.537207,0.540198,0.542141,0.526218,0.540667,0.534168,0.532774,0.528471,0.516442,0.504955
key,5.142606,5.188799,5.227722,5.207551,5.245104,5.199207,5.220508,5.236552,5.210538,5.244274
loudness,-11.294105,-11.492553,-11.502858,-12.012110,-11.831597,-12.069975,-12.347746,-12.602400,-12.949678,-13.168946
mode,0.700667,0.695997,0.690143,0.680375,0.666861,0.676703,0.677807,0.685887,0.681961,0.688553
speechiness,0.065988,0.069316,0.082132,0.066761,0.065191,0.062027,0.064189,0.064086,0.064879,0.068875
acousticness,0.446317,0.435572,0.432521,0.438737,0.427397,0.440263,0.439232,0.442796,0.451613,0.461619
instrumentalness,0.128307,0.138565,0.132285,0.154994,0.149328,0.154413,0.165507,0.168782,0.193736,0.178048


In [149]:
songs_per_year.columns

Index(['year', 'popularity', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'count',
       'duration_min'],
      dtype='object')

In [150]:
songs_per_year = songs_per_year.astype({'year': str})

In [151]:
alt.Chart(songs_per_year).mark_bar().encode(
    alt.X("year"),
    alt.Y("count")
)

alt.Chart(...)

In [108]:
songs_per_year.head()

,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,count,duration_min
0,1980,7.937137,0.519827,0.537207,5.142606,-11.294105,0.700667,0.065988,0.446317,0.128307,0.226420,0.579184,120.502448,243072.785499,3.859723,27902,4.051213
1,1981,8.046343,0.522204,0.540198,5.188799,-11.492553,0.695997,0.069316,0.435572,0.138565,0.227462,0.575045,120.959868,238512.530947,3.861316,25980,3.975209
2,1982,8.037993,0.529009,0.542141,5.227722,-11.502858,0.690143,0.082132,0.432521,0.132285,0.225734,0.572118,120.643812,239252.861822,3.863477,25373,3.987548
3,1983,8.095217,0.517713,0.526218,5.207551,-12.012110,0.680375,0.066761,0.438737,0.154994,0.221899,0.556607,119.864369,245990.748879,3.861658,26991,4.099846
4,1984,8.452725,0.519658,0.540667,5.245104,-11.831597,0.666861,0.065191,0.427397,0.149328,0.225375,0.549966,120.269657,242215.771837,3.874035,29159,4.036930


In [152]:
# Create the initial scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=songs_per_year['year'],
    y=songs_per_year['duration_ms'],
    #mode='markers',
    name='Duration (ms)'
))

fig.add_trace(go.Scatter(
    x=songs_per_year_month['year_month'],
    y=songs_per_year['duration_ms'],
    mode='markers',
    name='Duration (ms) - markers'
))

# Define the dropdown menu options
dropdown_options = [
    {'label': 'Duration (ms)', 'value': 'duration_ms'},
    {'label': 'Popularity', 'value': 'popularity'},
    {'label': 'Danceability', 'value': 'danceability'},
    {'label': 'Energy', 'value': 'energy'},
    {'label': 'Key', 'value': 'key'},
    {'label': 'Loudness', 'value': 'loudness'},
    {'label': 'Mode', 'value': 'mode'},
    {'label': 'Speechiness', 'value': 'speechiness'},
    {'label': 'Acousticness', 'value': 'acousticness'},
    {'label': 'Instrumentalness', 'value': 'instrumentalness'},
    {'label': 'Liveness', 'value': 'liveness'},
    {'label': 'Valence', 'value': 'valence'},
    {'label': 'Tempo', 'value': 'tempo'},
    {'label': 'Time Signature', 'value': 'time_signature'},
    {'label': 'Count', 'value': 'count'},
    {'label': 'Duration (min)', 'value': 'duration_min'}
]

# Create the dropdown menu
dropdown_menu = go.layout.Updatemenu(
    buttons=list([
        dict(
            args=[
                {'y': [songs_per_year[option['value']]]}, 
                {'yaxis': {'title': option['label'].capitalize()}}
            ],
            label=option['label'],
            method='update'
        )
        for option in dropdown_options
    ]),
    direction='down',
    showactive=True,
)

# Update the layout with dropdown menu and initial y-axis title
fig.update_layout(
    height = 800,
    updatemenus=[dropdown_menu],
    yaxis_title='Duration (ms)',
    xaxis_title='year',
    title='Development of features',
    showlegend=True
)

fig.show()

In [110]:
songs_per_year_genres = songs_per_year.merge(by_isrc_year, on='year', how='right')

#Create the initial scatter plot for genre 'pop'
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=songs_per_year_genres['year'],
    y=songs_per_year_genres['duration_ms'],
    #mode='line',
    name='pop',
    visible=True
))

# Add separate scatter plots for each genre
genres = ['rock', 'classical', 'german', 'jazz', 'folk', 'punk', 'metal', 'country', 'french']

for genre in genres:
    fig.add_trace(go.Scatter(
        x=songs_per_year_genres['year'],
        y=songs_per_year_genres['duration_ms'],
        #mode='line',
        name=genre,
        visible=False
    ))

# Create the dropdown menu for y-axis selection
dropdown_options = ['duration_ms', 'popularity', 'danceability', 'energy', 'key', 'loudness',
                    'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                    'valence', 'tempo', 'duration_min']

dropdown_menu = go.layout.Updatemenu(
    buttons=list([
        dict(
            args=[{'y': [songs_per_year_genres[option]]}],
            #label=option.capitalize(),
            method='update'
        )
        for option in dropdown_options
    ]),
    direction='down',
    showactive=True,
)

# Create the legend to toggle genre visibility
legend = dict(
    title='Genre',
    orientation='h',
    y=1.1,
    x=0.5,
    xanchor='center',
    bgcolor='rgba(255, 255, 255, 0.5)'
)

# Update the layout with dropdown menu, legend, and initial y-axis title
fig.update_layout(
    height = 800,
    updatemenus=[dropdown_menu],
    legend=legend,
    yaxis_title='Duration (ms)',
    title='Song Duration by Genre',
    xaxis_title='Year',
    showlegend=True
)

# Add custom buttons to toggle genre visibility in the legend
for i, genre in enumerate(genres):
    button = dict(
        label=genre.capitalize(),
        method='update',
        args=[{'visible': [True if j == i+1 else False for j in range(len(genres)+1)]}]
    )
    fig.update_traces(showlegend=True if genre == 'pop' else False, selector=dict(name=genre))
    fig.update_layout({'updatemenus': [{'buttons': [button]}]})

#fig.show()

## Development of genres of the years

In [111]:
features = sorted_genres[:10]

In [112]:
dev_genres_df = by_isrc_year.reset_index()

In [113]:
dev_genres_df

,year,pop,rock,german,punk,metal,country,blues,hard-rock,soul,british
0,1980,11523.0,8459.0,2248.0,2165.0,737.0,1804.0,1231.0,974.0,1391.0,917.0
1,1981,10277.0,8302.0,2471.0,2455.0,823.0,1711.0,1068.0,910.0,1359.0,690.0
2,1982,10359.0,7874.0,2185.0,2417.0,803.0,44.0,994.0,1034.0,1367.0,831.0
3,1983,10033.0,8292.0,3433.0,2366.0,1114.0,1483.0,971.0,1150.0,1034.0,630.0
4,1984,12202.0,8837.0,3047.0,2419.0,1532.0,1265.0,914.0,1328.0,835.0,825.0
5,1985,11372.0,8721.0,3443.0,2251.0,1612.0,1640.0,753.0,1135.0,809.0,730.0
6,1986,11430.0,9258.0,3824.0,2322.0,1807.0,1628.0,865.0,1182.0,766.0,983.0
7,1987,12310.0,10706.0,4458.0,2510.0,2293.0,1876.0,1279.0,1417.0,1014.0,1316.0
8,1988,13809.0,11045.0,6465.0,2736.0,2380.0,1930.0,1722.0,1221.0,1186.0,1515.0
9,1989,15447.0,12441.0,6178.0,2634.0,2660.0,2165.0,2414.0,1308.0,1338.0,2095.0


In [114]:
# Create the initial scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dev_genres_df['year'],
    y=dev_genres_df['pop'],
    #mode='markers',
    name='Pop'
))

# Create the dropdown menu
dropdown_menu = go.layout.Updatemenu(
    buttons=list([
        dict(
            args=[
                {'y': [dev_genres_df[option]]}, 
                {'yaxis': {'title': option.capitalize()}}
            ],
            label=option,
            method='update'
        )
        for option in features
    ]),
    direction='down',
    showactive=True,
)

# Update the layout with dropdown menu and initial y-axis title
fig.update_layout(
    height = 800,
    updatemenus=[dropdown_menu],
    yaxis_title='pop',
    xaxis_title='year',
    title='Development of genres'
)

fig.show()

## Add the genres to the features

In [115]:
df.head()

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,year_month
0,AEA040700577,"['j-pop', 'jazz', 'pop']",Bala Wala Chi,Ziad Rahbani,Houdou Nisbi,1985-01-01,day,spotify:track:0fylgLeNObjVvwhd8caHqX,0fylgLeNObjVvwhd8caHqX,NaN,...,0.0418,0.952,0.382,0.2370,0.371,157.168,207560,3,1985,1985.0
1,AEA040700578,"['groove', 'j-pop', 'jazz', 'pop']",Houdou Nisbi,Ziad Rahbani,Houdou Nisbi,1985-01-01,day,spotify:track:0yMFpBNCYXqwwOAg23bC8a,0yMFpBNCYXqwwOAg23bC8a,NaN,...,0.0308,0.814,0.951,0.0827,0.495,128.386,272240,3,1985,1985.0
2,AEA040700579,"['groove', 'j-pop', 'jazz', 'pop']",Nafs Al Sheghlat,Ziad Rahbani,Houdou Nisbi,1985-01-01,day,spotify:track:6G8l1kI8QlTD0UDIak5F8H,6G8l1kI8QlTD0UDIak5F8H,NaN,...,0.0449,0.954,0.887,0.2710,0.304,82.630,160093,4,1985,1985.0
3,AEA040700580,"['groove', 'j-pop', 'jazz', 'pop']",Yalla Kichou Barra,Ziad Rahbani,Houdou Nisbi,1985-01-01,day,spotify:track:21g76Lq5Jg4QvfTDvi4PlH,21g76Lq5Jg4QvfTDvi4PlH,NaN,...,0.0341,0.430,0.838,0.1240,0.656,102.485,155667,4,1985,1985.0
4,AEA040700581,"['groove', 'j-pop', 'jazz', 'pop', 'pop']",Ma Tfel,Ziad Rahbani,Houdou Nisbi,1985-01-01,day,spotify:track:0pKxrkFh8fxPKpkO29MYmi,0pKxrkFh8fxPKpkO29MYmi,NaN,...,0.0436,0.749,0.941,0.1150,0.752,150.656,265173,4,1985,1985.0


In [116]:
scaler = MinMaxScaler()
by_isrc_scaled = by_isrc.copy()
by_isrc_scaled.set_index('isrc', inplace=True)
by_isrc_scaled = by_isrc_scaled[['danceability', 'energy',
                       'key', 'loudness', 'mode', 'speechiness',
                       'acousticness', 'instrumentalness', 'liveness',
                       'valence', 'time_signature', 'tempo']]
by_isrc_scaled = pd.DataFrame(scaler.fit_transform(by_isrc_scaled), columns=by_isrc_scaled.columns, index = by_isrc_scaled.index)
by_isrc_scaled.reset_index(inplace=True)

In [117]:
by_isrc_genres = ct.merge(by_isrc_scaled, on=['isrc'], how='right')

In [118]:
by_isrc_genres.head()

,isrc,acoustic,afrobeat,alt-rock,alternative,ambient,black-metal,bluegrass,blues,brazil,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,tempo
0,AEA040700577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.818182,0.620601,1.0,0.043182,0.955823,0.382,0.2370,0.371,0.6,0.638375
1,AEA040700578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.636364,0.618011,0.0,0.031818,0.817269,0.951,0.0827,0.495,0.6,0.521470
2,AEA040700579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.272727,0.578544,1.0,0.046384,0.957831,0.887,0.2710,0.304,0.8,0.335621
3,AEA040700580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.454545,0.626845,1.0,0.035227,0.431727,0.838,0.1240,0.656,0.8,0.416267
4,AEA040700581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.454545,0.631776,0.0,0.045041,0.752008,0.941,0.1150,0.752,0.8,0.611925


## Create Radar Chart for the top 10 genres

In [119]:
sorted_genres[:10]

['pop',
 'rock',
 'german',
 'punk',
 'metal',
 'country',
 'blues',
 'hard-rock',
 'soul',
 'british']

In [159]:
fig = go.Figure()
for genre in sorted_genres[:10]:
    df_copy = by_isrc_genres.copy()
    df_copy = df_copy[df_copy[genre] == 1]
    df_copy = df_copy[['danceability', 'energy',
                       'key', 'loudness', 'mode', 'speechiness',
                       'acousticness', 'instrumentalness', 'liveness',
                       'valence', 'time_signature', 'tempo', 'danceability']]
    df_radar = pd.DataFrame(df_copy.mean(), columns=['mean'])
    df_radar.reset_index(inplace=True)
    df_radar.rename(columns={"index":"feature"}, inplace=True)

    fig.add_trace(go.Scatterpolar(
        r = df_radar['mean'],
        theta = df_radar['feature'],
        mode = 'lines',
        fill = 'none',
        name = genre
    ))

In [158]:
fig.update_layout(
    height = 1000
)
fig.show()

In [156]:
df_radar

,feature,mean
0,danceability,0.417908
1,energy,0.417698
2,key,0.460657
3,loudness,0.671141
4,mode,0.711356
5,speechiness,0.063828
6,acousticness,0.557675
7,instrumentalness,0.309827
8,liveness,0.221145
9,valence,0.397537
